In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
def processTrips(pid, records):
    import os
    os.environ["PROJ_LIB"] = 'C:\\Users\\sonali\\Anaconda3\\Library\\share'
    import csv
    import pyproj
    import shapely.geometry as geom
    import geopandas as gpd
    import fiona
    import fiona.crs
    import rtree

    neighbors = gpd.read_file('neighborhoods.geojson').to_crs(fiona.crs.from_epsg(2263))

    index = rtree.Rtree()
    for idx,geometry in enumerate(neighbors.geometry):
        index.insert(idx, geometry.bounds)

    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)

    if pid==0:
        next(records)

    reader = csv.reader(records)
    for row in reader:
        try:
            p = geom.Point(proj(float(row[5]), float(row[4])))
            for idx in index.intersection((p.x, p.y, p.x, p.y)):
                if neighbors.geometry[idx].contains(p):
                    borough = neighbors.borough[idx]
            if borough:
                q = geom.Point(proj(float(row[3]), float(row[2])))
                for idx in index.intersection((p.x, p.y, p.x, p.y)):
                    if neighbors.geometry[idx].contains(p):
                         neighborhood = neighbors.neighborhood[idx]
            if neighborhood and borough:
                yield((neighborhood,borough),1)
        except:
            pass

rdd = sc.textFile('yellow.csv')
result5 =rdd.mapPartitionsWithIndex(processTrips)\
              .reduceByKey(lambda x,y: x+y)\
              .map(lambda x: (x[0][1], (x[1],x[0][0]))).groupByKey()\
              .map(lambda x: (x[0],sorted(x[1],reverse= True)))\
              .map(lambda x: (x[0],x[1][:3]))

result5.take(5)


[('Manhattan',
  [(55792, 'Upper East Side'),
   (49865, 'Midtown'),
   (38621, 'Upper West Side')]),
 ('Brooklyn',
  [(7269, 'Williamsburg'),
   (2396, 'Bedford-Stuyvesant'),
   (2280, 'Greenpoint')]),
 ('Queens',
  [(3802, 'LaGuardia Airport'),
   (3621, 'Long Island City'),
   (3571, 'Astoria')]),
 ('Bronx', [(344, 'Mott Haven'), (232, 'Longwood'), (193, 'Kingsbridge')]),
 ('Staten Island', [(7, 'Concord'), (5, 'West Brighton'), (5, 'South Beach')])]